In [140]:
import pandas as pd

markets_info_df = pd.read_parquet("epl_markets.parquet")
all_histories = pd.read_parquet("token_histories.parquet")
markets_info_df.dtypes

winner                  string[python]
loser                   string[python]
is_draw                        boolean
question                string[python]
condition_id            string[python]
question_id             string[python]
id                      string[python]
description             string[python]
end_date_iso            string[python]
uma_end_data            string[python]
volume                         Float64
closed                         boolean
first_token_id          string[python]
first_token_outcome     string[python]
first_token_price              Float64
second_token_id         string[python]
second_token_outcome    string[python]
second_token_price             Float64
game_start_time         datetime64[ns]
dtype: object

In [141]:
markets_info_df = markets_info_df.assign(
    match_id=markets_info_df[["winner", "loser", "game_start_time"]]
    .agg(frozenset, axis=1)
    .apply(hash)
)

In [142]:
traded_df = markets_info_df[markets_info_df.volume > 0.01].drop_duplicates()
markets_info_traded_df_draws = (
    traded_df[traded_df.is_draw]
    .drop(columns=[c for c in traded_df.columns if "outcome" in c])
    .rename(columns={c: c.replace("first", "draw_yes") for c in traded_df.columns})
    .rename(columns={c: c.replace("second", "draw_no") for c in traded_df.columns})
    .drop(
        columns=[
            "is_draw",
            "question",
            "condition_id",
            "question_id",
            "id",
            "description",
            "end_date_iso",
            "uma_end_data",
        ]
    )
)
markets_info_traded_df_draws

,winner,loser,volume,closed,draw_yes_token_id,draw_yes_token_price,draw_no_token_id,draw_no_token_price,game_start_time,match_id
2,Newcastle United,Tottenham Hotspur,1648.157327,True,4968230879603371948866041796987077791752868535...,0.0,6486150380988866473748206016838559835962169397...,1.0,2024-09-01 12:30:00,-7021113643698515774
5,Chelsea,Crystal Palace,10562.379214,True,6628081043611276835629741132699692180128963103...,1.0,2038204275565301344807054547965085333069777021...,0.0,2024-09-01 12:30:00,976617573959895639
8,Manchester City,West Ham United,4246.952856,True,4652444992635632642529930521199192482880908236...,0.0,5519563340748631936641826338054313097421920479...,1.0,2024-08-31 16:30:00,243350930839179468
11,Aston Villa,Leicester City,6130.092364,True,7198006125524017729020999112949591754130072873...,0.0,1108105314871755573169486973066833564111072150...,1.0,2024-08-31 14:00:00,-759681699818020003
13,Bournemouth,Everton,2343.687726,True,6979466265655015180736494637283075984297819167...,0.0,9201488080747150865847341565897594405713204637...,1.0,2024-08-31 14:00:00,7114991348099034180
15,Fulham,Ipswich Town,1958.268762,True,1056239115982502641842514644390919491396338064...,1.0,6474528836620100582363221559257100699919142728...,0.0,2024-08-31 14:00:00,-7686705775813752617
17,Nottingham Forest,Wolverhampton Wanderers,2678.878368,True,1922636093149757117752916299704362461806023417...,1.0,1072383393901019078745413673212653811002433746...,0.0,2024-08-31 14:00:00,-6606115192345121424
25,Arsenal,Brighton & Hove Albion,8872.87221,True,3282101053200081500732589041854931815637585813...,1.0,5646917549123102449163419527608818678170441410...,0.0,2024-08-31 11:30:00,-6178097404848359776
26,Brentford,Southampton,502.707692,True,3102412140979869086108516105922538862811635984...,0.0,1071771006010210786837770353476734639789497155...,1.0,2024-08-31 14:00:00,-1960004041559828337
30,Brentford,Liverpool,9199.510944,True,4937274880027886183807479622344736653709925151...,0.0,8933757800840528696678082589188567084286739295...,1.0,2024-08-25 15:30:00,-6372488427790714442


In [143]:
traded_wl_df = (
    traded_df[~traded_df.is_draw]
    .drop(columns=[c for c in traded_df.columns if "outcome" in c])
    .rename(columns={c: c.replace("first", "first_win_yes") for c in traded_df.columns})
    .rename(columns={c: c.replace("second", "first_win_no") for c in traded_df.columns})
    .drop(
        columns=[
            "is_draw",
            "question",
            "condition_id",
            "question_id",
            "id",
            "description",
            "end_date_iso",
            "uma_end_data",
        ]
    )
)

traded_wl_df_switched = (
    traded_df[~traded_df.is_draw]
    .drop(columns=[c for c in traded_df.columns if "outcome" in c])
    .rename(columns={"winner": "loser", "loser": "winner"})
    .rename(
        columns={c: c.replace("first", "second_win_yes") for c in traded_df.columns}
    )
    .rename(
        columns={c: c.replace("second", "second_win_no") for c in traded_df.columns}
    )
    .drop(
        columns=[
            "is_draw",
            "question",
            "condition_id",
            "question_id",
            "id",
            "description",
            "end_date_iso",
            "uma_end_data",
        ]
    )
)

traded_wl_df_switched

,loser,winner,volume,closed,second_win_yes_token_id,second_win_yes_token_price,second_win_no_token_id,second_win_no_token_price,game_start_time,match_id
0,Manchester United,Liverpool,27321.177281,True,1070268098614301483769400035048192945868028748...,0.0,4641946105864093247918510751580466890037570902...,1.0,2024-09-01 15:00:00,8403341662787711466
1,Crystal Palace,Chelsea,16465.558516,True,2240836493145815788231779528861370764772082802...,0.0,9112607163262702891896524070679080530791126243...,1.0,2024-09-01 12:30:00,976617573959895639
3,Chelsea,Crystal Palace,74848.61748,True,8473141659136127648919471372081023410636831516...,0.0,7168237386744928574044957256855095466455619768...,1.0,2024-09-01 12:30:00,976617573959895639
4,Newcastle United,Tottenham Hotspur,16197.184165,True,3902396297125861646265048524779791779669292294...,1.0,6390143151867843714655104214159797394603648846...,0.0,2024-09-01 12:30:00,-7021113643698515774
6,Tottenham Hotspur,Newcastle United,5338.051969,True,5462649995618436636028509551748704905681350869...,0.0,5130458727082798673488813540528413902741629816...,1.0,2024-09-01 12:30:00,-7021113643698515774
...,...,...,...,...,...,...,...,...,...,...
214,Liverpool,Aston Villa,62.783485,True,6039475446325558590610830792923439094235664256...,1.0,7478682910843262796119864286568811611031017045...,0.0,2023-09-03 11:00:00,-4527743939016230220
215,Arsenal,Manchester United,24.285713,True,5719121657815100684863998168411340463668945474...,1.0,4633691986224411951686891585684086012813294716...,0.0,2023-09-03 13:30:00,-4306841848665369009
219,Arsenal,Bournemouth,37.138885,True,3476227583566235893385207197809170546483891999...,1.0,6626472848259693355623768601741507876320467959...,0.0,2023-09-30 14:00:00,352885978626819290
220,Crystal Palace,Wolverhampton Wanderers,0.068964,True,3991272135719335455425689967467591073248100202...,1.0,4095702638071914562064741827563675366981703730...,0.0,2023-09-03 11:00:00,1866254063103686471


In [144]:
traded_wl_df

,winner,loser,volume,closed,first_win_yes_token_id,first_win_yes_token_price,first_win_no_token_id,first_win_no_token_price,game_start_time,match_id
0,Manchester United,Liverpool,27321.177281,True,1070268098614301483769400035048192945868028748...,0.0,4641946105864093247918510751580466890037570902...,1.0,2024-09-01 15:00:00,8403341662787711466
1,Crystal Palace,Chelsea,16465.558516,True,2240836493145815788231779528861370764772082802...,0.0,9112607163262702891896524070679080530791126243...,1.0,2024-09-01 12:30:00,976617573959895639
3,Chelsea,Crystal Palace,74848.61748,True,8473141659136127648919471372081023410636831516...,0.0,7168237386744928574044957256855095466455619768...,1.0,2024-09-01 12:30:00,976617573959895639
4,Newcastle United,Tottenham Hotspur,16197.184165,True,3902396297125861646265048524779791779669292294...,1.0,6390143151867843714655104214159797394603648846...,0.0,2024-09-01 12:30:00,-7021113643698515774
6,Tottenham Hotspur,Newcastle United,5338.051969,True,5462649995618436636028509551748704905681350869...,0.0,5130458727082798673488813540528413902741629816...,1.0,2024-09-01 12:30:00,-7021113643698515774
...,...,...,...,...,...,...,...,...,...,...
214,Liverpool,Aston Villa,62.783485,True,6039475446325558590610830792923439094235664256...,1.0,7478682910843262796119864286568811611031017045...,0.0,2023-09-03 11:00:00,-4527743939016230220
215,Arsenal,Manchester United,24.285713,True,5719121657815100684863998168411340463668945474...,1.0,4633691986224411951686891585684086012813294716...,0.0,2023-09-03 13:30:00,-4306841848665369009
219,Arsenal,Bournemouth,37.138885,True,3476227583566235893385207197809170546483891999...,1.0,6626472848259693355623768601741507876320467959...,0.0,2023-09-30 14:00:00,352885978626819290
220,Crystal Palace,Wolverhampton Wanderers,0.068964,True,3991272135719335455425689967467591073248100202...,1.0,4095702638071914562064741827563675366981703730...,0.0,2023-09-03 11:00:00,1866254063103686471


In [145]:
merged_duped = (
    traded_wl_df.merge(
        traded_wl_df_switched,
        on=["winner", "loser", "game_start_time", "match_id"],
        suffixes=("_first_win", "_second_win"),
    )
    .merge(
        markets_info_traded_df_draws,
        on=["winner", "loser", "game_start_time", "match_id"],
        suffixes=("", "_draw"),
    )
    .rename(columns={"winner": "first", "loser": "second"})
    .groupby("match_id")
    .first()
    .reset_index()
)
merged_duped

,match_id,first,second,volume_first_win,closed_first_win,first_win_yes_token_id,first_win_yes_token_price,first_win_no_token_id,first_win_no_token_price,game_start_time,...,second_win_yes_token_id,second_win_yes_token_price,second_win_no_token_id,second_win_no_token_price,volume,closed,draw_yes_token_id,draw_yes_token_price,draw_no_token_id,draw_no_token_price
0,-7686705775813752617,Fulham,Ipswich Town,5815.749542,True,5390684612591871180226940463614602007632608985...,0.0,4835867034043260323210602260980380172342860099...,1.0,2024-08-31 14:00:00,...,2724863763841420578046708779868589387441936666...,0.0,4592496941083629569447042476295961754389643201...,1.0,1958.268762,True,1056239115982502641842514644390919491396338064...,1.0,6474528836620100582363221559257100699919142728...,0.0
1,-7207986795286300592,Bournemouth,Newcastle United,6817.51712,True,5595597303624263375013541904246077929556506181...,0.0,2829599343436084726317107655436762981197686206...,1.0,2024-08-25 13:00:00,...,1082024554349514253531564589217953799162453000...,0.0,4432888292968002860833730739498129959429415973...,1.0,4162.556595,True,9048290216715822669728345412856006694560460510...,1.0,2081943794877508791078019192487212357316597380...,0.0
2,-7021113643698515774,Newcastle United,Tottenham Hotspur,16197.184165,True,3902396297125861646265048524779791779669292294...,1.0,6390143151867843714655104214159797394603648846...,0.0,2024-09-01 12:30:00,...,5462649995618436636028509551748704905681350869...,0.0,5130458727082798673488813540528413902741629816...,1.0,1648.157327,True,4968230879603371948866041796987077791752868535...,0.0,6486150380988866473748206016838559835962169397...,1.0
3,-6606115192345121424,Nottingham Forest,Wolverhampton Wanderers,1996.301956,True,6154942560231987257537066294648112889077743278...,0.0,7274828458575385276549632588018871878040247012...,1.0,2024-08-31 14:00:00,...,7650721475605775229306239491570036659123722307...,0.0,9334483916931061512989620164357410658582777708...,1.0,2678.878368,True,1922636093149757117752916299704362461806023417...,1.0,1072383393901019078745413673212653811002433746...,0.0
4,-6372488427790714442,Brentford,Liverpool,10460.211613,True,8058812937056295301077754963716989842392959855...,0.0,4586745120104545026109118389516543779698202352...,1.0,2024-08-25 15:30:00,...,6181134725108988943382990745734226687444082523...,1.0,2689521720270122027066650781604176657297852439...,0.0,9199.510944,True,4937274880027886183807479622344736653709925151...,0.0,8933757800840528696678082589188567084286739295...,1.0
5,-6178097404848359776,Arsenal,Brighton & Hove Albion,38583.198239,True,6111503160491403443221654004462481775096015381...,0.0,5891194826219718921556265951457901437359904302...,1.0,2024-08-31 11:30:00,...,1116610378897274981341652849429823672930913311...,0.0,5259322497049373785466208713290228068678828929...,1.0,8872.87221,True,3282101053200081500732589041854931815637585813...,1.0,5646917549123102449163419527608818678170441410...,0.0
6,-5668478559215340077,Fulham,Leicester City,3539.832769,True,4607962402989626140266501134480645068086366987...,1.0,7261539017555037054997917278338131112479553989...,0.0,2024-08-24 14:00:00,...,4469108998165111035747799236588079997245140576...,0.0,5424410382959485122279389019281489307327875564...,1.0,2098.449725,True,8573518884612686090908478468464167480657353847...,0.0,1815059595045740714942503862018463711066903613...,1.0
7,-4473749050102683384,Everton,Tottenham Hotspur,2620.177761,True,6275796669559057519575423296233233326165712844...,0.0,8691539046046940277665499791939596124790931338...,1.0,2024-08-24 14:00:00,...,4917496998749688653456658639021768943769564153...,1.0,7883479697323373245952680051157073912273260113...,0.0,8131.359812,True,1030756381874894962188194491483407194908508542...,0.0,1065955351846421313987627768369814483760100781...,1.0
8,-1960004041559828337,Brentford,Southampton,8893.260638,True,9285165882976373321289124694825334422477391015...,1.0,6983060241552922298703023478823448877784249474...,0.0,2

In [154]:
pre_game_timedelta = pd.Timedelta(minutes=5)


def get_price_at_time(info_df, history_df, token_col, pre_game_timedelta):
    joined_gametime_prices = (
        history_df.reset_index(names=["token_id", "timestamp"])
        .merge(
            info_df[
                [
                    "game_start_time",
                    token_col,
                ]
            ],
            left_on="token_id",
            right_on=token_col,
        )
        .assign(
            timestamp=lambda x: pd.to_datetime(x.timestamp),
        )
        .assign(pre_game_time=lambda x: x.game_start_time - pre_game_timedelta)
    )

    price_at_time = (
        joined_gametime_prices.loc[
            (joined_gametime_prices.timestamp < joined_gametime_prices.pre_game_time)
        ]
        .groupby("token_id")
        .last()
    )
    return price_at_time.price


token_cols_list = [
    "first_win_yes_token_id",
    "first_win_no_token_id",
    "second_win_yes_token_id",
    "second_win_no_token_id",
    "draw_yes_token_id",
    "draw_no_token_id",
]

df = merged_duped.assign(
    **{
        s[:-4]+ "_pre_price": get_price_at_time(
            merged_duped, all_histories, s, pre_game_timedelta
        ).values
        for s in token_cols_list
    }
)
df

,match_id,first,second,volume_first_win,closed_first_win,first_win_yes_token_id,first_win_yes_token_price,first_win_no_token_id,first_win_no_token_price,game_start_time,...,draw_yes_token_id,draw_yes_token_price,draw_no_token_id,draw_no_token_price,first_win_yes_toke_pre_price,first_win_no_toke_pre_price,second_win_yes_toke_pre_price,second_win_no_toke_pre_price,draw_yes_toke_pre_price,draw_no_toke_pre_price
0,-7686705775813752617,Fulham,Ipswich Town,5815.749542,True,5390684612591871180226940463614602007632608985...,0.0,4835867034043260323210602260980380172342860099...,1.0,2024-08-31 14:00:00,...,1056239115982502641842514644390919491396338064...,1.0,6474528836620100582363221559257100699919142728...,0.0,0.905,0.500,0.490,0.690,0.275,0.830
1,-7207986795286300592,Bournemouth,Newcastle United,6817.51712,True,5595597303624263375013541904246077929556506181...,0.0,2829599343436084726317107655436762981197686206...,1.0,2024-08-25 13:00:00,...,9048290216715822669728345412856006694560460510...,1.0,2081943794877508791078019192487212357316597380...,0.0,0.265,0.825,0.265,0.595,0.170,0.750
2,-7021113643698515774,Newcastle United,Tottenham Hotspur,16197.184165,True,3902396297125861646265048524779791779669292294...,1.0,6390143151867843714655104214159797394603648846...,0.0,2024-09-01 12:30:00,...,4968230879603371948866041796987077791752868535...,0.0,6486150380988866473748206016838559835962169397...,1.0,0.305,0.270,0.370,0.570,0.280,0.735
3,-6606115192345121424,Nottingham Forest,Wolverhampton Wanderers,1996.301956,True,6154942560231987257537066294648112889077743278...,0.0,7274828458575385276549632588018871878040247012...,1.0,2024-08-31 14:00:00,...,1922636093149757117752916299704362461806023417...,1.0,1072383393901019078745413673212653811002433746...,0.0,0.705,0.490,0.310,0.510,0.225,0.750
4,-6372488427790714442,Brentford,Liverpool,10460.211613,True,8058812937056295301077754963716989842392959855...,0.0,4586745120104545026109118389516543779698202352...,1.0,2024-08-25 15:30:00,...,4937274880027886183807479622344736653709925151...,0.0,8933757800840528696678082589188567084286739295...,1.0,0.175,0.435,0.110,0.415,0.265,0.755
5,-6178097404848359776,Arsenal,Brighton & Hove Albion,38583.198239,True,6111503160491403443221654004462481775096015381...,0.0,5891194826219718921556265951457901437359904302...,1.0,2024-08-31 11:30:00,...,3282101053200081500732589041854931815637585813...,1.0,5646917549123102449163419527608818678170441410...,0.0,0.510,0.615,0.245,0.425,0.275,0.730
6,-5668478559215340077,Fulham,Leicester City,3539.832769,True,4607962402989626140266501134480645068086366987...,1.0,7261539017555037054997917278338131112479553989...,0.0,2024-08-24 14:00:00,...,8573518884612686090908478468464167480657353847...,0.0,1815059595045740714942503862018463711066903613...,1.0,0.200,0.610,0.405,0.195,0.265,0.720
7,-4473749050102683384,Everton,Tottenham Hotspur,2620.177761,True,6275796669559057519575423296233233326165712844...,0.0,8691539046046940277665499791939596124790931338...,1.0,2024-08-24 14:00:00,...,1030756381874894962188194491483407194908508542...,0.0,1065955351846421313987627768369814483760100781...,1.0,0.340,0.690,0.185,0.615,0.295,0.735
8,-1960004041559828337,Brentford,Southampton,8893.260638,True,9285165882976373321289124694825334422477391015...,1.0,6983060241552922298703023478823448877784249474...,0.0,2024-08-31 14:00:00,...,3102412140979869086108516105922538862811635984...,0.0,1071771006010210786837770353476734639789497155...,1.0,0.715,0.695,0.315,0.665,0.250,0.745
9,-1958410684441444657,Fulham,Manchester United,55641.874117,True,1315385407057815291697000886516025911268371559...,0.0,1406282590488127830418252672774515413854682441...,1.0,2024-08-16 19:00:00,...,4574822642040335406924607620918373394202085737...,0.0,9413900031581212950670543634633325437183025849...,1.0,0.400,0.595,0.245,0.755,0.170,0.770


In [111]:
import plotly.express as px

px.scatter(procesed_df, x="price", y="first_token_price")

In [8]:
import numpy as np

(
    np.round(procesed_df.price, 0) == np.round(procesed_df.first_token_price, 0)
).sum() / len(procesed_df)

np.float64(0.717948717948718)